In [1]:
from linear import run_lin
from non_linear import run_nonlin
from conv import run_conv

## linear

In [2]:
run_lin(2000, 4000)

restoring network from: ./tmp/lowpass/linear/2000/model.ckpt
starting from epoch: 4000

	                         mse                    rmse                                 std            
	        training  validation    training  validation    training  validation   reference     runtime
epoch: 4000      0.00419     0.00227     0.06472     0.04761     0.06502     0.04762     0.10262        -1.0


In [3]:
run_lin(1000, 4000)

restoring network from: ./tmp/lowpass/linear/1000/model.ckpt
starting from epoch: 4000

	                         mse                    rmse                                 std            
	        training  validation    training  validation    training  validation   reference     runtime
epoch: 4000      0.00306     0.00429     0.05527     0.06548     0.05570     0.06549     0.11196        -1.0


In [9]:
run_lin(500, 4000)

restoring network from: ./tmp/lowpass/linear/500/model.ckpt
starting from epoch: 4000

	                         mse                    rmse                                 std            
	        training  validation    training  validation    training  validation   reference     runtime
epoch: 4000      0.00334     0.00452     0.05776     0.06720     0.05803     0.06721     0.11230        -1.0


--------------

## non-linear

In [8]:
run_nonlin(500, 8000)

restoring network from: ./tmp/lowpass/nonlinear/500/model.ckpt
starting from epoch: 8000

	                         mse                    rmse                                 std            
	        training  validation    training  validation    training  validation   reference     runtime
epoch: 8000      0.00558     0.00299     0.07467     0.05467     0.07505     0.05468     0.10128        -1.0


In [4]:
run_nonlin(1000, 4000)

restoring network from: ./tmp/lowpass/nonlinear/1000/model.ckpt
starting from epoch: 4000

	                         mse                    rmse                                 std            
	        training  validation    training  validation    training  validation   reference     runtime
epoch: 4000      0.00480     0.00314     0.06926     0.05608     0.06959     0.05608     0.11188        -1.0


----

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))